Compares the differences in the Nash Bargaining Issue, grouped by issue and p-values. 

This notebook relies on an database connection. 

Set the variable `model_run_ids` either in this file, in the settings as default or in the .env (recommended). 

In [1]:
import sqlite3

import pandas as pd

from decideanalyses import settings
%matplotlib inline
from IPython.core.display import display, HTML

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)


In [2]:
database_connection = sqlite3.connect(settings.DATABASE_URL)
model_run_ids = settings.MODEL_RUN_IDS
model_run_id_param =  ','.join(model_run_ids)


In [3]:
sql = """SELECT a.p                              as p,
       a.issue                          as issue,
       a.repetion || '-' || a.iteration as pointer,
       a.numerator / a.denominator      AS nbs
FROM (SELECT sum(ai.position * ai.power * ai.salience) AS numerator,
             sum(ai.salience * ai.power)               AS denominator,
             r.pointer                                 AS repetion,
             i2.pointer                                AS iteration,
             m.p,
             i.name                                    as issue
      FROM actorissue ai
               LEFT JOIN issue i ON ai.issue_id = i.id
               LEFT JOIN actor a ON ai.actor_id = a.id
               LEFT JOIN iteration i2 ON ai.iteration_id = i2.id
               LEFT JOIN repetition r ON i2.repetition_id = r.id
               LEFT JOIN modelrun m ON r.model_run_id = m.id
      WHERE ai.type = 'before'
        AND i2.pointer = 9
        AND m.id IN (%s)
      GROUP BY m.id, r.id, i2.id, i.id) a""" % model_run_id_param

In [4]:
df = pd.read_sql(
    sql=sql, 
    con=database_connection, 
    index_col=['p'],
    columns=['issue']
)


# Covariance
Covariance of the NBS between the issues, grouped by p-value 

In [5]:
for p in sorted(set(df.index)):
    x = df.loc[p].pivot(index='pointer', columns='issue', values='nbs').cov().round(5)
    display(HTML('<h1>p={}</h1>'.format(str(p))))
    display(x)


issue,adapt_finance,adapt_inst,adapt_legal,amb2050,amb2100,differentiation,eaa,finance_vol,finance_who,ld,legal,mrv,progress
issue,,,,,,,,,,,,,
adapt_finance,0.21972,-0.25007,0.04165,-0.70949,-0.05279,0.26573,0.35538,0.13620,0.10118,-0.42273,-0.33946,-0.70733,0.41959
adapt_inst,-0.25007,0.28461,-0.04740,0.80749,0.06008,-0.30243,-0.40447,-0.15501,-0.11516,0.48112,0.38635,0.80504,-0.47754
adapt_legal,0.04165,-0.04740,0.00789,-0.13448,-0.01001,0.05037,0.06736,0.02582,0.01918,-0.08013,-0.06434,-0.13407,0.07953
amb2050,-0.70949,0.80749,-0.13448,2.29102,0.17045,-0.85806,-1.14757,-0.43980,-0.32672,1.36502,1.09616,2.28406,-1.35489
amb2100,-0.05279,0.06008,-0.01001,0.17045,0.01268,-0.06384,-0.08538,-0.03272,-0.02431,0.10156,0.08155,0.16993,-0.10080
differentiation,0.26573,-0.30243,0.05037,-0.85806,-0.06384,0.32137,0.42980,0.16472,0.12237,-0.51124,-0.41055,-0.85545,0.50745
eaa,0.35538,-0.40447,0.06736,-1.14757,-0.08538,0.42980,0.57481,0.22030,0.16365,-0.68374,-0.54906,-1.14408,0.67866
finance_vol,0.13620,-0.15501,0.02582,-0.43980,-0.03272,0.16472,0.22030,0.08443,0.06272,-0.26204,-0.21043,-0.43847,0.26010
finance_who,0.10118,-0.11516,0.01918,-0.32672,-0.02431,0.12237,0.16365,0.06272,0.04659,-0.19467,-0.15632,-0.32573,0.19322


issue,adapt_finance,adapt_inst,adapt_legal,amb2050,amb2100,differentiation,eaa,finance_vol,finance_who,ld,legal,mrv,progress
issue,,,,,,,,,,,,,
adapt_finance,0.05394,-0.42312,0.00435,-0.07386,0.27443,-0.76974,0.41970,0.38303,0.26104,-0.27879,0.31843,-1.02461,-0.18497
adapt_inst,-0.42312,3.31927,-0.03415,0.57943,-2.15284,6.03838,-3.29245,-3.00474,-2.04776,2.18701,-2.49800,8.03780,1.45106
adapt_legal,0.00435,-0.03415,0.00035,-0.00596,0.02215,-0.06213,0.03388,0.03092,0.02107,-0.02250,0.02570,-0.08271,-0.01493
amb2050,-0.07386,0.57943,-0.00596,0.10115,-0.37581,1.05409,-0.57475,-0.52452,-0.35747,0.38177,-0.43606,1.40311,0.25330
amb2100,0.27443,-2.15284,0.02215,-0.37581,1.39630,-3.91642,2.13544,1.94883,1.32815,-1.41847,1.62017,-5.21321,-0.94114
differentiation,-0.76974,6.03838,-0.06213,1.05409,-3.91642,10.98496,-5.98960,-5.46619,-3.72527,3.97859,-4.54434,14.62229,2.63975
eaa,0.41970,-3.29245,0.03388,-0.57475,2.13544,-5.98960,3.26585,2.98046,2.03122,-2.16934,2.47782,-7.97286,-1.43934
finance_vol,0.38303,-3.00474,0.03092,-0.52452,1.94883,-5.46619,2.98046,2.72001,1.85372,-1.97977,2.26129,-7.27614,-1.31356
finance_who,0.26104,-2.04776,0.02107,-0.35747,1.32815,-3.72527,2.03122,1.85372,1.26333,-1.34924,1.54110,-4.95878,-0.89521
